# Scheduling (2019 refactoring)

In [ ]:
import sys
import math
import random
import time
from ipywidgets import interact_manual, SelectMultiple
from matplotlib import pylab

sys.path.insert(0, "..")
from sdp_par_model import reports, config
from sdp_par_model.scheduling import graph, level_trace, scheduler
from sdp_par_model.parameters import definitions
from sdp_par_model.parameters.definitions import Telescopes, Pipelines, Constants, HPSOs
from sdp_par_model import config

## Overall observatory selection & capacities

In [ ]:
telescope = Telescopes.SKA1_Mid

# Telescope-specific system sizing
if telescope == Telescopes.SKA1_Low:
    total_flops = int(13.8 * Constants.peta) # FLOP/s
    buffer_size = int(46.0 * Constants.peta) # Byte
    hot_buffer_size = int(buffer_size / 2.7) # Byte
else:
    total_flops = int(12.1 * Constants.peta) # FLOP/s
    buffer_size = int(46.0 * Constants.peta) # Byte
    hot_buffer_size = int(buffer_size / 2.4) # Byte

# Common system sizing
ingest_rate = 0.45 * Constants.tera # Byte/s
cold_rate = ingest_rate + 0.65 * Constants.tera # Byte/s
hot_rate = 5.0 * Constants.tera # Byte/s
delivery_rate = int(100/8 * Constants.giga)  # Byte/s
lts_rate = delivery_rate

# Extra allowance for delivery
delivery_buffer_size = int(buffer_size / 30) # Byte
input_buffer_size = buffer_size - hot_buffer_size

## Read HPSO performance characteristics

Loads high performance science objective characteristics generated by the export notebook. This always picks up the latest file checked into Git.

In [ ]:
csv = reports.read_csv(reports.newest_csv(reports.find_csvs()))
csv = reports.strip_csv(csv)

## Determine computational capacity required for realtime processing

As SKA SDP needs to be able to change observation at arbitrary times, we need to always reserve enough computational resources to deal with the most expensive case. Here we figure this out automatically based on the calculated parameters.

In [ ]:
realtime_flops = 0
realtime_flops_hpso = None
for hpso in definitions.HPSOs.all_hpsos:
    if definitions.HPSOs.hpso_telescopes[hpso] != telescope:
        continue
    # Sum FLOP rates over involved real-time pipelines
    rt_flops = 0
    for pipeline in definitions.HPSOs.hpso_pipelines[hpso]:
        cfg_name = config.PipelineConfig(hpso=hpso, pipeline=pipeline).describe()
        flops = int(math.ceil(float(reports.lookup_csv(csv, cfg_name, 'Total Compute Requirement')) * definitions.Constants.peta))
        if pipeline in definitions.Pipelines.realtime:
            rt_flops += flops
    # Dominates?
    if rt_flops > realtime_flops:
        realtime_flops = rt_flops
        realtime_flops_hpso = hpso
        
# Show
print("Realtime processing requirements:")
batch_flops = total_flops - realtime_flops
print(" {:.3f} Pflop/s real-time (from {}), {:.3f} Pflop/s left for batch".format(
    realtime_flops / definitions.Constants.peta,
    realtime_flops_hpso, batch_flops / definitions.Constants.peta))

## Derive capacities

Now that we know the split between batch and realtime processing, we can formulate the capacity dictionary that will be used in scheduling.

In [ ]:
capacities = {
    graph.Resources.Observatory: 1,
    graph.Resources.BatchCompute: batch_flops,
    graph.Resources.RealtimeCompute: realtime_flops,
    graph.Resources.InputBuffer: input_buffer_size,
    graph.Resources.HotBuffer: hot_buffer_size,
    graph.Resources.OutputBuffer: delivery_buffer_size,
    graph.Resources.IngestRate: ingest_rate,
    graph.Resources.ColdBufferRate: cold_rate,
    graph.Resources.HotBufferRate: hot_rate,
    graph.Resources.DeliveryRate: delivery_rate,
    graph.Resources.LTSRate: lts_rate
}

# HACKs: Adjustments to make things work
if telescope == Telescopes.SKA1_Mid:
    capacities[graph.Resources.OutputBuffer] *= 3
    capacities[graph.Resources.DeliveryRate] *= 6
capacities[graph.Resources.HotBufferRate] *= 2

## Generate graph

Generate a sequence with all HPSOs appearing roughly as often as we expect them in a real-life schedule. We then shuffle this list and generate a (multi-)graph of tasks from it.

Note that in contrast to Francois' scheduler, the resource usage of every task is fixed up-front, therefore we need to declare certain key sizes here. Adjust as necessary in relation to the capacities (see below) to get the desired amount of parallelism between tasks.

In [ ]:
Tsequence = 20 * 24 * 3600
Tobs_min = 10 * 60

hpso_sequence, Tobs_sum = graph.make_hpso_sequence(telescope, Tsequence, Tobs_min, verbose=True)
print("{:.3f} d total".format(Tobs_sum / 3600 / 24))
random.shuffle(hpso_sequence)

In [ ]:
t = time.time()
nodes = graph.hpso_sequence_to_nodes(csv, hpso_sequence, capacities, Tobs_min)
print("Multi-graph has {} nodes (generation took {:.3f}s)".format(len(nodes), time.time()-t))

## Sanity-check Graph

We can do a number of consistency checks at this point: Clearly we should have enough capacity to run every task in isolation.

Furthermore, in order to keep up with observations we need to make sure that we are not over-using any resource on average. This is a pretty rough estimate of safety that especially under-estimates the cost of edges in high-pressure scenarios. For example, if somethings needs to be kept in the buffer for longer, it has a higher footprint than estimated here. Therefore especially the size of `input-buffer` and `output-buffer` should be quite generous here.

In [ ]:
import warnings
cost_sum = { cost : 0 for cost in capacities.keys() }
for task in nodes:
    for cost, amount in task.all_cost().items():
        assert cost in capacities, "No {} capacity defined, required by {}!".format(cost, task.name)
        assert amount <= capacities[cost], "Not enough {} capacity to run {} ({:g}<{:g}!)".format(
            cost, task.name, capacities[cost], amount)
        # Try to compute an average. Edges are the main wild-card here: We only know that they stay
        # around at least for the lifetime of the dependency *and* the longest dependent task.
        ttime = task.time
        if cost in task.edge_cost and len(task.rev_deps) > 0:
            ttime += max([d.time for d in task.rev_deps])
        cost_sum[cost] += ttime * amount
print("Best-case average loads:")
for cost in graph.Resources.All:
    unit, mult = graph.Resources.units[cost]
    avg = cost_sum[cost] / Tobs_sum
    cap = capacities[cost]
    print(" {}:\t{:.3f} {} ({:.1f}% of {:.3f} {})".format(cost, avg/mult, unit, avg/cap*100, cap/mult, unit))
    # Warn past 75%
    if avg > cap:
        print('Likely insufficient {} capacity!'.format(cost), file=sys.stderr,)

## Schedule tasks

Assign a task time to every node, and figure out resource usages and edge lengths along the way.

In [ ]:
t = time.time()
usage, task_time, task_edge_end_time = scheduler.schedule(nodes, capacities, verbose=False)
print("Scheduling took {:.3f}s".format(time.time() - t))
print("Observing efficiency: {:.1f}%".format(Tobs_sum / usage[graph.Resources.Observatory].end() * 100))

In [ ]:
trace_end = max(*task_edge_end_time.values())
pylab.figure(figsize=(16,40))
for n, cost in enumerate(graph.Resources.All):
    levels = usage[cost]
    avg = levels.average(0,trace_end)    
    unit, mult = graph.Resources.units[cost]
    pylab.subplot(len(usage), 1, n+1)
    pylab.step([0] + [ t/24/3600 for t in levels._trace.keys() ] + [trace_end],
               [0] + [ v/mult for v in  levels._trace.values() ] + [0],
               where='post')
    pylab.title("{}: {:.3f} {} average ({:.2f}%)".format(
        cost, avg/mult, unit, avg / capacities[cost] * 100))
    pylab.xlim((0, trace_end/24/3600)); pylab.xticks(range(int(trace_end)//24//3600+1))
    pylab.ylim((0, capacities[cost] / mult * 1.01))
    pylab.ylabel(unit)
pylab.xlabel("Days")
pylab.show()

## Efficiency calculations

We can play around with capacities and see how it affects overall efficiency. This takes quite a bit, so let's set up some multiprocessing infrastructure to take advantage of parallelism:

In [ ]:
def determine_efficiencies(cost_amounts):
    cost, amounts = cost_amounts
    # Make new HPSO sequence
    hpso_seq = list(hpso_sequence)
    random.shuffle(hpso_seq)
    # Schedule, collect efficiencies
    effs = []
    for a in amounts:
        cap = dict(capacities)
        cap[cost] = int(a)
        nodes = graph.hpso_sequence_to_nodes(csv, hpso_seq, cap, Tobs_min)
        try:
            usage, task_time, task_edge_end_time = scheduler.schedule(nodes, cap, verbose=False)
            effs.append(Tobs_sum / usage[graph.Resources.Observatory].end() * 100)
        except ValueError:
            effs.append(None)
    return effs
# Set up multiprocessing map. Fall back to standard map if the system doesn't support it (e.g. Windows)
try:
    import multiprocessing
    mp_pool = multiprocessing.Pool(multiprocessing.cpu_count() // 2)
    mp_map = mp_pool.map
except:
    print("Falling back to single-threaded map. This should work, but is very slow!", file=sys.stderr)
    mp_map = map

In [ ]:
import numpy
import matplotlib.lines

interesting_costs = [ cost for cost in graph.Resources.All
                      if cost not in [graph.Resources.RealtimeCompute,
                                      graph.Resources.IngestRate,
                                      graph.Resources.LTSRate ]]
@interact_manual(costs=SelectMultiple(options=graph.Resources.All, value=interesting_costs),
                 percent=(1,100,1), steps=(1,10,1), count=(1,100,1))
def test_sensitivity(costs=graph.Resources.All, percent=20, steps=5, count=12):
    graph_count = len(costs)
    pylab.figure(figsize=(8,graph_count*4))
    pylab.subplots_adjust(hspace=0.4)
    for graph_ix, cost in enumerate(costs):
        # Calculate efficiencies for different amounts
        amounts = capacities[cost] * (1+numpy.arange(-percent, percent+steps, steps)/100)
        effs = numpy.transpose(list(mp_map(determine_efficiencies, count * [(cost, amounts)])))
        # Filter out "None" values
        sel = numpy.all(effs != None, axis=1)
        amounts = amounts[sel]; effs = effs[sel]
        # Draw percentiles
        unit,mult = graph.Resources.units[cost]
        percents = [10,25,50,75,90]; styles = [':', '--', '-', '--', ':']
        percentiles = numpy.percentile(effs, percents, axis=1)
        pylab.subplot(graph_count, 1, graph_ix+1)
        for p, eff_ps, style in zip(percents, percentiles, styles):
            pylab.plot(amounts/mult, eff_ps, label="{}%".format(p), linestyle=style, color='blue')
        # Draw line for "default" value
        ymin, ymax = pylab.gca().get_ybound()
        if ymin < 80:
            ymin = 80; pylab.gca().set_ylim((80, ymax))
        pylab.gca().add_line(matplotlib.lines.Line2D([capacities[cost]/mult,capacities[cost]/mult], [ymin,ymax],
                                                     color='black', linestyle=':'))
        # Titles
        pylab.title('{} sensitivity'.format(cost)); pylab.xlabel("[{}]".format(unit))
        pylab.ylabel("Efficiency [%]")
        pylab.legend()